# 🏡 House Prices - Advanced Regression Techniques

このNotebookでは、Kaggleの「House Prices」コンペにおいて、  
LightGBMを用いた住宅価格予測モデルを構築し、提出データを作成します。

---

## 🎯 目的

- **住宅価格（SalePrice）を予測**する精度の高いモデルを構築
- データの構造を理解し、**前処理・特徴量選択・モデル評価**を丁寧に実施
- クロスバリデーションを活用して汎化性能を確認

---

## 🧠 構成・アプローチ

1. **データ前処理**  
   - 欠損値補完（カテゴリ型: `'None'`、数値型: `0`）  
   - One-hot encoding によるカテゴリ変換（`get_dummies`）

2. **目的変数の変換**  
   - `SalePrice` を log変換（`np.log1p`）し、正規性とRMSEの安定化を図る

3. **外れ値除去**  
   - `GrLivArea > 4600` の物件は明らかな高価格帯の外れ値として除外

4. **特徴量選択**  
   - LightGBMで学習 → 重要度上位40特徴量を選抜し再学習

5. **モデル学習 & 予測**  
   - LightGBMモデルを使用し、testデータを予測  
   - 逆log変換（`np.expm1`）を適用して `SalePrice` を復元

6. **提出ファイル生成**  
   - `submission.csv` を作成し、Kaggleへ提出

7. **クロスバリデーション（任意）**  
   - KFoldによるCVでモデルの汎化性能（RMSE）を確認

---

## 🔧 使用モデル・技術

- LightGBM（Gradient Boostingベースの回帰モデル）
- 特徴量重要度による変数選択
- クロスバリデーション（KFold, 5分割）
- 評価指標：RMSE（Root Mean Squared Error）

---

このNotebookは、基本的な前処理からモデル構築・評価・提出までを一貫して行えるように設計されています。


In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# データ読み込み
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

# SalePriceを取り出しておく
y_train = train_df['SalePrice']



In [2]:
# 'Id' は不要
train = train_df.drop(['Id', 'SalePrice'], axis=1)
test = test_df.drop(['Id'], axis=1)

# trainとtestを結合（同じ変換をかけるため）
all_data = pd.concat([train, test], axis=0)

# 欠損補完（シンプルな戦略）
for col in all_data.select_dtypes(include='object'):
    all_data[col] = all_data[col].fillna('None')

for col in all_data.select_dtypes(exclude='object'):
    all_data[col] = all_data[col].fillna(0)

# One-hot encoding
all_data_encoded = pd.get_dummies(all_data)

# train/testに分割し直す
X_train = all_data_encoded[:len(train_df)]
X_test = all_data_encoded[len(train_df):]


In [3]:
# log変換
y_log = np.log1p(y_train)

# reset_indexしてインデックス揃える
X_train = X_train.reset_index(drop=True)
y_log = y_log.reset_index(drop=True)

# 外れ値除去
outlier_index = X_train['GrLivArea'] > 4600
X_train_cleaned = X_train[~outlier_index].copy()
y_log_cleaned = y_log[~outlier_index].copy()



In [4]:
model_temp = LGBMRegressor(n_estimators=100, random_state=42)
model_temp.fit(X_train_cleaned, y_log_cleaned)

# 重要度上位40
feature_importance = model_temp.feature_importances_
feature_names = X_train_cleaned.columns
top_indices = np.argsort(feature_importance)[::-1][:40]
selected_features = feature_names[top_indices].tolist()




[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004118 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3479
[LightGBM] [Info] Number of data points in the train set: 1458, number of used features: 202
[LightGBM] [Info] Start training from score 12.024015


In [5]:
# 特徴量を絞る
X_train_top_clean = X_train_cleaned[selected_features]
X_test_top = X_test[selected_features]

# 最終モデル学習
model_final = LGBMRegressor(n_estimators=100, random_state=42)
model_final.fit(X_train_top_clean, y_log_cleaned)

# 予測（逆log変換）
y_test_pred_log = model_final.predict(X_test_top)
y_test_pred = np.expm1(y_test_pred_log)

# 提出ファイル作成
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': y_test_pred
})
submission.to_csv('submission.csv', index=False)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000316 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3076
[LightGBM] [Info] Number of data points in the train set: 1458, number of used features: 40
[LightGBM] [Info] Start training from score 12.024015


In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = []

for train_index, val_index in kf.split(X_train_top_clean):
    X_tr, X_val = X_train_top_clean.iloc[train_index], X_train_top_clean.iloc[val_index]
    y_tr, y_val = y_log_cleaned.iloc[train_index], y_log_cleaned.iloc[val_index]

    model_cv = LGBMRegressor(n_estimators=100, random_state=42)
    model_cv.fit(X_tr, y_tr)

    y_pred = model_cv.predict(X_val)
    rmse = mean_squared_error(np.expm1(y_val), np.expm1(y_pred), squared=False)
    rmse_scores.append(rmse)

print("CV RMSE scores:", [f"{s:.2f}" for s in rmse_scores])
print(f"CV RMSE avg: {np.mean(rmse_scores):.2f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000195 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2834
[LightGBM] [Info] Number of data points in the train set: 1166, number of used features: 40
[LightGBM] [Info] Start training from score 12.023362
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000190 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2837
[LightGBM] [Info] Number of data points in the train set: 1166, number of used features: 40
[LightGBM] [Info] Start training from score 12.026498
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000199 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug